In [22]:
import pickle
import nltk
import math

In [23]:
class vect:
    SCALAR = 1
    COSINE = 2
    JACCARD = 3

class Docs:

    def __init__(self,Lancaster=True,split=False):

        docs = []
        self.maxfreq = {}
        MotsVides = nltk.corpus.stopwords.words('english')
        self.Lancaster = Lancaster
        self.split = split
        self.files = ["TP2data/D1.txt","TP2data/D2.txt","TP2data/D3.txt","TP2data/D4.txt","TP2data/D5.txt","TP2data/D6.txt"]
        file = ""
        my_dict = {}
        for i in range(len(self.files)):
            file = open(self.files[i]).read()
            file = file.replace("\n","")

            if not split :
                Termes = file.split()
            if split:

                ExpReg = nltk.RegexpTokenizer('(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*') # \d : équivalent à [0-9] >>> 
                Termes = ExpReg.tokenize(file) 


            if Lancaster :
                TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
                Lancaster = nltk.LancasterStemmer()
                TermesNormalisation = [Lancaster.stem(terme) for terme in TermesSansMotsVides]
            else :
                TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
                porter = nltk.PorterStemmer()
                TermesNormalisation = [porter.stem(terme) for terme in TermesSansMotsVides]


            for term in TermesNormalisation:
                
                if (term,i) in my_dict:
                    my_dict[(term,i)] += 1
                else:
                    my_dict[(term,i)] = 1

        self.dict = my_dict


        for keys in self.dict.keys():
            if keys[1] not in docs:
                docs.append(keys[1])
            if keys[1] not in self.maxfreq.keys():
                self.maxfreq[keys[1]] = 0
            if self.dict[keys] > self.maxfreq[keys[1]]:
                self.maxfreq[keys[1]] = self.dict[keys]

        self.N = len(docs)
    
    def rebuild(self,Lancaster=True,split=False):
        docs = []
        self.maxfreq = {}
        MotsVides = nltk.corpus.stopwords.words('english')
        self.Lancaster = Lancaster
        self.split = split
        
        file = ""
        my_dict = {}
        for i in range(len(self.files)):
            file = open(self.files[i]).read()
            file = file.replace("\n","")


            if not split :
                Termes = file.split()
            if split:
                ExpReg = nltk.RegexpTokenizer('(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*') # \d : équivalent à [0-9] >>> 
                Termes = ExpReg.tokenize(file) 


            if Lancaster :
                TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
                Lancaster = nltk.LancasterStemmer()
                TermesNormalisation = [Lancaster.stem(terme) for terme in TermesSansMotsVides]
            else :
                TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
                porter = nltk.PorterStemmer()
                TermesNormalisation = [porter.stem(terme) for terme in TermesSansMotsVides]

            for term in TermesNormalisation:
                
                if (term.lower(),i) in my_dict:
                    my_dict[(term,i)] += 1
                else:
                    my_dict[(term,i)] = 1

        self.dict = my_dict


        for keys in self.dict.keys():
            if keys[1] not in docs:
                docs.append(keys[1])
            if keys[1] not in self.maxfreq.keys():
                self.maxfreq[keys[1]] = 0
            if self.dict[keys] > self.maxfreq[keys[1]]:

                self.maxfreq[keys[1]] = self.dict[keys]

        self.N = len(docs)
        self.DictWeight()
    def normalize(self,text):
        if self.Lancaster :
            Lancaster = nltk.LancasterStemmer()
            text = Lancaster.stem(text)
        else :
            porter = nltk.PorterStemmer()
            text = porter.stem(text)
        return text
    def splitter(self,text):
        if not self.split:
            return text.split(" ")
        else:
            ExpReg = nltk.RegexpTokenizer('(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*') # \d : équivalent à [0-9] >>> 
            Termes = ExpReg.tokenize(text) 
            return Termes
    def calcWeight(self,word,doc):

        inDocs = []

        for keys in self.dict.keys():
            if keys[0] == word:
                if keys[1] not in inDocs:
                    inDocs.append(keys[1])

        # nombre de documents contenant le mot
        Ni = len(inDocs)

        try:
            tf = self.dict[(word,doc)]/self.maxfreq[doc]
        except:
            tf = 0
            
        #idf = math.log10((N/Ni)+1)
        try:
            idf = math.log10((self.N/Ni)+1)
        except:
            idf = 0

        return(tf*idf)

    def DictWeight(self):

        self.TDFP = [(word,doc,self.dict[(word,doc)],self.calcWeight(word,doc)) for word,doc in self.dict.keys()]
        self.DTFP = [(doc,word,freq,weight) for word,doc,freq,weight in self.TDFP]
        self.TDFP = sorted(self.TDFP, key=lambda x: x[0])

    def MV(self,termes,typeV = vect.SCALAR):
        rsv = {}
        termes = self.splitter(termes)
        normterms = []
        for term in termes:
            normterms.append(self.normalize(term))

        for doc in range(self.N):
            QD = 0
            v2 = 0
            w2 = 0
            for term in normterms:
                if (term.lower(),doc) in self.dict.keys():
                    weight = self.calcWeight(term,doc)
                    QD+= weight
                    w2+= math.pow(weight,2)
                    v2+=1
            try:
                if typeV == vect.COSINE:
                    QD = QD / (v2*w2)
                if typeV == vect.JACCARD:
                    QD = QD / (w2+v2 - QD)
            except ZeroDivisionError:
                QD = 0


            rsv[doc] = QD

        rsv = dict(sorted(rsv.items(), key=lambda item: item[1],reverse=True))
        return rsv

        
d = Docs(Lancaster=False,split=True)

d.DictWeight()


<>:26: SyntaxWarning: invalid escape sequence '\.'
<>:77: SyntaxWarning: invalid escape sequence '\.'
<>:123: SyntaxWarning: invalid escape sequence '\.'
<>:26: SyntaxWarning: invalid escape sequence '\.'
<>:77: SyntaxWarning: invalid escape sequence '\.'
<>:123: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Mekki\AppData\Local\Temp\ipykernel_13044\2324064935.py:26: SyntaxWarning: invalid escape sequence '\.'
  ExpReg = nltk.RegexpTokenizer('(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*') # \d : équivalent à [0-9] >>>
C:\Users\Mekki\AppData\Local\Temp\ipykernel_13044\2324064935.py:77: SyntaxWarning: invalid escape sequence '\.'
  ExpReg = nltk.RegexpTokenizer('(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*') # \d : équivalent à [0-9] >>>
C:\Users\Mekki\AppData\Local\Temp\ipykernel_13044\2324064935.py:123: SyntaxWarning: invalid escape sequence '\.'
  ExpReg = nltk.RegexpTokenizer('(?:[

In [24]:
import tkinter as tk
from tkinter import ttk

def search():
    global stemming

    Lancaster=stemming.get()

    text = query_text.get()
    table.delete(*table.get_children())
    
    if Lancaster :
        Lancaster = nltk.LancasterStemmer()
        text = Lancaster.stem(text)
    else :
        porter = nltk.PorterStemmer()
        text = porter.stem(text)

    if index_var.get() == 0:
        data = d.TDFP
    else:
        data = d.DTFP

    for terme in data:

        if type(terme[0]) is str:
            if text.lower() in terme[0].lower():
                table.insert("", "end", values=[terme[0],terme[1],terme[2],terme[3]])
        else:
            if text.lower() in terme[1].lower():
                table.insert("", "end", values=[terme[0],terme[1],terme[2],terme[3]])
    # Add your search and processing logic here
    pass

def on_token_change(*args):
    d.rebuild(Lancaster=stemming.get(),split=token.get())
    search()

def on_index_change(*args):
    search()
def show_match():
    text = query_text.get()
    table.delete(*table.get_children())

    if selected_option.get() == "SCALAR":
        rsv = d.MV(text,vect.SCALAR)
    if selected_option.get() == "COSINE":
        rsv = d.MV(text,vect.COSINE)
    if selected_option.get() == "JACCARD":
        rsv = d.MV(text,vect.JACCARD)

    for doc in rsv.keys():
        table.insert("", "end", values=[doc,"",rsv[doc],""])
        
# Create the main application window
root = tk.Tk()
root.title("Text Processing Application")

# Search Query Text Box and Search Button
query_label = tk.Label(root, text="Search Query:")
query_label.grid(row=0, column=0)
query_text = tk.Entry(root, width=40)
query_text.grid(row=0, column=1)
search_button = tk.Button(root, text="Search", command=search)
search_button.grid(row=0, column=2)

# Border around "Processing" section
processing_label_frame = ttk.LabelFrame(root, text="Processing")
processing_label_frame.grid(row=1, column=2, padx=10, pady=10)

# Radio buttons for Processing
stemming = tk.IntVar()
stemming.trace("w", on_token_change)

token = tk.IntVar()
token.trace("w", on_token_change)

tokenization_radiobutton = tk.Checkbutton(processing_label_frame, text="Tokenization", variable=token)
porter_stemmer_radiobutton = tk.Checkbutton(processing_label_frame, text="Lancaster Stemmer", variable=stemming)
porter_stemmer_radiobutton.select()
tokenization_radiobutton.grid(row=0, column=0)
porter_stemmer_radiobutton.grid(row=1, column=0)

# Border around "Index" section
index_label_frame = ttk.LabelFrame(root, text="Index")
index_label_frame.grid(row=1, column=0, padx=10, pady=10)

# Radio buttons for Index

index_var = tk.IntVar()
index_var.trace("w", on_index_change)

docs_per_term_radiobutton = tk.Radiobutton(index_label_frame, text="DOCS per TERM", variable=index_var, value=0)
terms_per_doc_radiobutton = tk.Radiobutton(index_label_frame, text="TERMS per DOC", variable=index_var, value=1)
docs_per_term_radiobutton.grid(row=1, column=0)
terms_per_doc_radiobutton.grid(row=2, column=0)

# Table to display data
table_frame = ttk.Frame(root)
table_frame.grid(row=2, column=0, columnspan=3, pady=10)
table = ttk.Treeview(table_frame, columns=("1", "2", "3", "4"))
table.heading("#1", text="terme")
table.heading("#2", text="document")
table.heading("#3", text="freq")
table.heading("#4", text="Score")
table['show'] = 'headings'
# Vertical scrollbar for the table
table_scrollbar = ttk.Scrollbar(table_frame, orient="vertical", command=table.yview)
table.configure(yscrollcommand=table_scrollbar.set)

# Grid layout for the table and scrollbar
table.grid(row=0, column=0, sticky="nsew")
table_scrollbar.grid(row=0, column=1, sticky="ns")


# Make the table resizable
table_frame.grid_rowconfigure(0, weight=1)
table_frame.grid_columnconfigure(0, weight=1)

# Border around "Processing" section
Matching_label_frame = ttk.LabelFrame(root, text="Matching")
Matching_label_frame.grid(row=1, column=3, padx=10, pady=10)

# Create a StringVar to hold the selected option
selected_option = tk.StringVar()

# Create a Combobox
combo = ttk.Combobox(Matching_label_frame, textvariable=selected_option, state='readonly')
combo['values'] = ('SCALAR','JACCARD', 'COSINE')  # Set the available choices
combo.set('SCALAR')  # Set the default choice
combo.grid(row=0, column=0)
show_matching_button = tk.Button(Matching_label_frame, text="Show Matching", command=show_match)
show_matching_button.grid(row=1, column=0)

for dat in d.TDFP:
    table.insert("", "end", values=[dat[0],dat[1],dat[2],dat[3]])


root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\APPS\python\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Mekki\AppData\Local\Temp\ipykernel_13044\1199361307.py", line 37, in on_token_change
    search()
  File "C:\Users\Mekki\AppData\Local\Temp\ipykernel_13044\1199361307.py", line 10, in search
    table.delete(*table.get_children())
                  ^^^^^^^^^^^^^^^^^^^^
  File "d:\APPS\python\Lib\tkinter\ttk.py", line 1195, in get_children
    self.tk.call(self._w, "children", item or '') or ())
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
_tkinter.TclError: invalid command name ".!frame.!treeview"
